In [15]:
import pandas as pd
import os
import numpy as np

In [16]:
datasets = []
allsites = []
for file in os.listdir('data'):
    dataset = pd.read_csv(f'data/{file}')
    datasets.append(dataset)
    sites = set(dataset['sample.samplingPoint.notation'])
    allsites.append(sites)
data = pd.concat(datasets)
sites = set.intersection(*allsites)

In [17]:
features = ['Alkalinity to pH 4.5 as CaCO3', 'Ammonia un-ionised as N', 'Nitrite as N', 'Nitrogen, Total Oxidised as N', 'Orthophosphate, reactive as P', 'Oxygen, Dissolved as O2','Oxygen, Dissolved, % Saturation', 'pH']

In [18]:
y_features = [f'Y_{x}' for x in features]

In [19]:
df = data

In [20]:
df['date'] = pd.to_datetime(df['sample.sampleDateTime']).dt.date

In [21]:
df = df[['sample.samplingPoint.notation','date', 'determinand.definition', 'result']]
df = df.rename(columns = {'sample.samplingPoint.notation': 'location', 'date': 'date', 'determinand.definition':'feature', 'determinand.notation':'result'})

In [22]:
df

,location,date,feature,result
0,TH-PBNR0001,2020-01-08,"Magnesium, Dissolved",8.5
1,TH-PBNR0001,2020-01-08,"Iron, Dissolved",260.0
2,TH-PBNR0001,2020-01-08,"Oxygen, Dissolved, % Saturation",87.0
3,TH-PBNR0001,2020-01-08,"Carbon, Organic, Dissolved as C :- {DOC}",9.5
4,TH-PBNR0001,2020-01-08,"Calcium, Dissolved",65.0
...,...,...,...,...
115789,TH-WWWWW999,2011-12-13,Crocidolite :- {Blue Asbestos},0.0
115790,TH-WWWWW999,2011-12-13,Asbestos : Anthophyllite,0.0
115791,TH-WWWWW999,2011-12-13,Asbestos,1.0
115792,TH-WWWWW999,2011-12-13,Asbestos : Tremolite -Tremolite,0.0


In [28]:
tables = []
l = len(sites)
i = 0
for site in sites:
    print(f'{i}/{l}')
    print(site)
    i += 1
    temp = df[df['location'] == site]
    table = pd.pivot_table(temp, values = 'result', index = ['date'], columns = 'feature', aggfunc = 'sum')
    try:
        table = table[features]
    except:
        continue
    table['date'] = table.index
    table = table.dropna(axis = 0, how = 'any')
    print(table)   
    tables.append(table)
    
    

0/46
TH-PUTR0104
147
concatenating
            Alkalinity to pH 4.5 as CaCO3  Ammonia un-ionised as N  \
2003-12-17                          198.0                  0.00008   
2007-01-23                          201.0                  0.00067   
2007-02-19                          226.0                  0.00039   
2007-03-20                          205.0                  0.00046   
2007-04-20                          223.0                  0.00057   
...                                   ...                      ...   
2023-04-05                          230.0                  0.00037   
2023-05-22                          240.0                  0.00048   
2023-06-19                          240.0                  0.00057   
2023-07-28                          250.0                  0.00059   
2023-08-11                          250.0                  0.00047   

            Nitrite as N  Nitrogen, Total Oxidised as N  \
2003-12-17        0.0472                           12.1   
2007-0

In [29]:
agg = pd.concat(tables)
agg

,Alkalinity to pH 4.5 as CaCO3,Ammonia un-ionised as N,Nitrite as N,"Nitrogen, Total Oxidised as N","Orthophosphate, reactive as P","Oxygen, Dissolved as O2","Oxygen, Dissolved, % Saturation",pH,date,Y_Alkalinity to pH 4.5 as CaCO3,Y_Ammonia un-ionised as N,Y_Nitrite as N,"Y_Nitrogen, Total Oxidised as N","Y_Orthophosphate, reactive as P","Y_Oxygen, Dissolved as O2","Y_Oxygen, Dissolved, % Saturation",Y_pH
2007-01-23,201.0,0.00067,0.0040,14.30,0.155,11.40,92.4,7.60,2007-01-23,226.0,0.00039,0.0040,12.40,0.020,11.50,103.0,7.84
2007-02-19,226.0,0.00039,0.0040,12.40,0.020,11.50,103.0,7.84,2007-02-19,205.0,0.00046,0.0040,13.00,0.020,11.90,99.9,8.02
2007-03-20,205.0,0.00046,0.0040,13.00,0.020,11.90,99.9,8.02,2007-03-20,223.0,0.00057,0.0096,10.30,0.020,11.50,103.6,8.03
2007-04-20,223.0,0.00057,0.0096,10.30,0.020,11.50,103.6,8.03,2007-04-20,217.0,0.00078,0.0164,9.18,0.020,10.40,105.2,7.97
2007-05-24,217.0,0.00078,0.0164,9.18,0.020,10.40,105.2,7.97,2007-05-24,250.0,0.00078,0.0222,8.47,0.020,8.87,84.8,8.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-02-08,225.0,0.00040,0.0111,6.65,0.020,11.20,90.0,8.13,2013-02-08,230.0,0.00064,0.0141,6.19,0.025,9.67,84.2,8.21
2013-08-19,213.0,0.00079,0.0138,4.53,0.056,8.77,87.4,8.23,2013-08-19,212.0,0.00064,0.0120,4.36,0.075,9.52,89.0,8.21
2013-09-10,212.0,0.00064,0.0120,4.36,0.075,9.52,89.0,8.21,2013-09-10,219.0,0.00067,0.0132,4.35,0.096,8.46,80.0,8.22
2013-11-14,222.0,0.00048,0.0123,6.09,0.043,10.40,88.0,8.01,2013-11-14,226.0,0.00042,0.0167,6.00,0.038,11.10,90.0,8.17


In [30]:
sites

{'TH-PCHR0030',
 'TH-PEVR0006',
 'TH-PEVR0052',
 'TH-PGWU0567',
 'TH-PGWU0658',
 'TH-PGWU0659',
 'TH-PGWU0660',
 'TH-PGWU0661',
 'TH-PGWU0662',
 'TH-PGWU0986',
 'TH-PGWU1577',
 'TH-PGWU1615',
 'TH-PKER0025',
 'TH-PKER0052',
 'TH-PKER0058',
 'TH-PKER0063',
 'TH-PKER0160',
 'TH-PLDR0017',
 'TH-PLDR0029',
 'TH-PLDR0073',
 'TH-PLDR9999',
 'TH-PTAR0022',
 'TH-PTAR0028',
 'TH-PTAR9999',
 'TH-PTHR0008',
 'TH-PTHR0012',
 'TH-PTHR0054',
 'TH-PTHR0075',
 'TH-PTHR0082',
 'TH-PTHR0099',
 'TH-PTHR0107',
 'TH-PTHR0111',
 'TH-PTHR0124',
 'TH-PUTR0036',
 'TH-PUTR0039',
 'TH-PUTR0080',
 'TH-PUTR0104',
 'TH-PWER0008',
 'TH-PWER0027',
 'TH-PWER0030',
 'TH-PWER0033',
 'TH-PWER9999',
 'TH-PWRR0021',
 'TH-PWRR0023',
 'TH-PWRR0025',
 'TH-PWRR0028'}

In [31]:
agg.sort_values(by = 'date')

,Alkalinity to pH 4.5 as CaCO3,Ammonia un-ionised as N,Nitrite as N,"Nitrogen, Total Oxidised as N","Orthophosphate, reactive as P","Oxygen, Dissolved as O2","Oxygen, Dissolved, % Saturation",pH,date,Y_Alkalinity to pH 4.5 as CaCO3,Y_Ammonia un-ionised as N,Y_Nitrite as N,"Y_Nitrogen, Total Oxidised as N","Y_Orthophosphate, reactive as P","Y_Oxygen, Dissolved as O2","Y_Oxygen, Dissolved, % Saturation",Y_pH
2000-01-06,222.0,0.00100,0.0630,10.90,0.429,12.51,103.4,8.10,2000-01-06,231.0,0.00200,0.0690,9.29,0.717,10.84,92.0,8.10
2000-01-10,185.0,0.00100,0.0110,10.80,0.020,11.90,98.8,8.20,2000-01-10,206.0,0.00100,0.0160,10.20,0.029,11.56,99.9,8.20
2000-01-10,189.0,0.00100,0.0090,9.76,0.020,11.80,94.3,8.20,2000-01-10,211.0,0.00100,0.0130,9.45,0.059,11.63,99.8,8.30
2000-01-10,186.0,0.00100,0.0090,9.56,0.020,11.80,95.0,8.20,2000-01-10,188.0,0.00100,0.0180,8.82,0.089,11.59,99.8,8.20
2000-01-12,79.0,0.00100,0.0570,4.31,1.360,10.95,90.8,7.60,2000-01-12,76.0,0.00100,0.0360,4.56,1.720,12.10,100.8,7.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-11,200.0,0.00077,0.0100,6.50,0.052,9.84,97.0,8.08,2023-07-11,200.0,0.00078,0.0083,6.20,0.061,10.60,104.7,8.18
2023-07-14,250.0,0.00041,0.0210,7.80,0.031,10.40,97.6,7.79,2023-07-14,250.0,0.00032,0.0360,8.10,0.025,8.61,79.3,7.71
2023-07-19,220.0,0.00025,0.0250,1.50,0.230,5.03,51.4,7.45,2023-07-19,220.0,0.00023,0.0120,1.90,0.200,5.58,55.1,7.47
2023-07-24,220.0,0.00024,0.0120,8.40,0.034,9.42,87.8,7.57,2023-07-24,220.0,0.00040,0.0110,8.20,0.032,9.95,97.0,7.57


In [32]:
agg.to_csv('var_output.csv')

In [33]:
agg.to_excel('var_output.xlsx')